### 필수과제1
- medium, source에 대한 예산을 집행한다고 했을 때 -> 크게 medium, source 예산을 가정해서 만들고 (예산 금액은 임의로 10억 미만 선으로) 
- 해당 예산을 스프레드해서 예산 컬럼을 만들고 -> 컬럼 기준으로 효율성을 분석해 보자
- 예산과 pageviews, hits, visits에 따른 효율성을 측정하는 지표를 만들어 주세요!
- plotly를 통해 해당 효율성 지표의 시각화를 보여주시면 됩니다.
    - 시각화는 line, sunbrust 등등 가장 효율적으로 잘 보여줄 수 있는 시각화 방법이면 됩니다!
    - 어떤 traffic이 가장 효율적이고, 비효율적인지?
    - 두 개를 계산하여서 시각적으로 보여주시면 됩니다!

# 📊 필수과제 1: 트래픽 예산 효율성 분석

## ✅ 과제 목표
- `source`, `medium` 조합별로 예산을 집행했다고 가정하고,
- `pageviews`, `hits`, `visits` 기준 성과를 통해 **1000원당 효율성 지표**를 계산한다.
- Plotly 등 시각화 도구를 통해 **가장 효율적 / 비효율적 트래픽**을 시각적으로 비교한다.


### ✅ Step 1. 데이터 준비

- 기준 데이터: `agg_df_sm` (날짜별 집계된 트래픽 데이터)
- 포함 컬럼: `source`, `medium`, `pageviews`, `hits`, `visits`

```python
agg_df_sm.head()


In [ ]:
import pandas as pd
import numpy as np
import json

# 데이터 로드
df = pd.read_csv('/Users/parkjunhyeok/Desktop/BDA_/지표/data/ga.csv')

# JSON 컬럼 파싱
df['totals'] = df['totals'].apply(json.loads)
df['trafficSource'] = df['trafficSource'].apply(json.loads)

# 수치형 값 추출 (결측 시 0)
df['pageviews'] = df['totals'].apply(lambda x: int(x.get('pageviews', 0)))
df['hits'] = df['totals'].apply(lambda x: int(x.get('hits', 0)))
df['visits'] = df['totals'].apply(lambda x: int(x.get('visits', 0)))

# 트래픽 정보 추출 및 정제
df['source'] = df['trafficSource'].apply(lambda x: x.get('source', '(not set)')).str.strip()
df['medium'] = df['trafficSource'].apply(lambda x: x.get('medium', '(not set)')).str.strip()
df['date'] = df['date'].astype(str)


### ✅ Step 2. 트래픽 집계 테이블 생성

In [4]:
# (source, medium) 단위로 집계
agg_df_sm = df.groupby(['source', 'medium'], as_index=False)[['pageviews', 'hits', 'visits']].sum()


### ✅ Step 3. 예산 스프레드 (총 10억 원)

In [5]:
# 총 예산 가정
total_budget = 1_000_000_000
np.random.seed(42)

# 랜덤 비율 생성 → 전체 합이 1이 되도록 정규화
rand_weights = np.random.rand(len(agg_df_sm))
rand_weights /= rand_weights.sum()

# 각 조합별 예산 분배
agg_df_sm['budget'] = (rand_weights * total_budget).astype(int)


### ✅ Step 4. 효율성 지표 계산
예산에 1000원당 성과로 했음

In [6]:
# 1000원당 성과
agg_df_sm['pageviews_per_1000won'] = agg_df_sm['pageviews'] / (agg_df_sm['budget'] / 1000)
agg_df_sm['hits_per_1000won'] = agg_df_sm['hits'] / (agg_df_sm['budget'] / 1000)
agg_df_sm['visits_per_1000won'] = agg_df_sm['visits'] / (agg_df_sm['budget'] / 1000)

# 시각화를 위한 식별자
agg_df_sm['combo'] = agg_df_sm['medium'] + ' / ' + agg_df_sm['source']


### ✅ Step 5. 효율 상위/하위 10개 트래픽 추출

In [7]:
top10 = agg_df_sm.sort_values('pageviews_per_1000won', ascending=False).head(10)
bottom10 = agg_df_sm.sort_values('pageviews_per_1000won', ascending=True).head(10)


### ✅ Step 6. 시각화 (matplotlib 버전 – 정적 그래프)

In [8]:
import plotly.express as px

# 상위 10개 효율적인 트래픽 plotly 시각화
fig_top = px.bar(
    top10,
    x='pageviews_per_1000won',
    y='combo',
    orientation='h',
    color='pageviews_per_1000won',
    color_continuous_scale='Blues',
    title='Top 10 효율적인 트래픽 (pageviews per 1000₩)'
)
fig_top.update_layout(yaxis=dict(autorange="reversed"))

# 하위 10개 비효율적인 트래픽 plotly 시각화
fig_bottom = px.bar(
    bottom10,
    x='pageviews_per_1000won',
    y='combo',
    orientation='h',
    color='pageviews_per_1000won',
    color_continuous_scale='Reds',
    title='Bottom 10 비효율적인 트래픽 (pageviews per 1000₩)'
)
fig_bottom.update_layout(yaxis=dict(autorange="reversed"))

fig_top.show()
fig_bottom.show()

# 📊 트래픽 효율성 분석 결과 요약

## 📈 Top 10 효율적인 트래픽

### ✅ 어떤 조합이 가장 효율적이었나?
- **referral / mall.googleplex.com**
- **organic / google**
- **(none) / (direct)**

이 조합들은 **1000원당 평균 3~5회 이상의 페이지뷰(pageviews)**를 발생시킨 유입 경로입니다.

### 🔍 왜 효율적일까?
- `organic / google`과 `(direct)` 트래픽은 대부분 **유료 광고 없이도 들어온 자연 유입**입니다.
- 이는 사용자들이 **브랜드를 인지하고 직접 찾아왔거나**, **검색으로 쉽게 발견된다는 의미**입니다.
- → **브랜드 인지도, 콘텐츠 신뢰도, SEO 효과**가 작용했을 가능성이 큽니다.

### 💡 인사이트
- 이들 트래픽은 **광고비 없이도 높은 성과**를 냈기 때문에,
- 예산을 적게 쓰더라도 **지속 유지하거나, 더 강화할 가치가 있는 유입 경로**입니다.

---

## 📉 Bottom 10 비효율적인 트래픽

### ⚠️ 어떤 조합이 비효율적이었나?
- **referral / trainup.withgoogle.com**
- **referral / reddit.com**
- **referral / quora.com**
- 등 다양한 리퍼럴 기반 유입 경로들

이 조합들은 **1000원당 평균 0.001 pageviews 이하**의 매우 낮은 성과를 보였습니다.

### 🔍 왜 비효율적일까?
- 대부분의 유입이 특정 외부 사이트에서 발생했지만,
- 실제로는 **사용자가 거의 페이지를 보지 않거나**, **바로 이탈했을 가능성**이 있습니다.
- 예산이 투입되었음에도 **성과(조회수)가 거의 없었다**는 뜻입니다.

### ⚠️ 인사이트
- 이런 트래픽은 **광고 효율이 매우 낮은 채널**로 판단되며,
- 향후 **광고 중단 또는 개선 테스트(A/B Test)**가 필요합니다.
- 또는 **랜딩 페이지 품질이나 타겟팅 자체가 부적절했을 가능성**도 고려해볼 수 있습니다.

---

## 🧠 요약 정리

| 항목 | 요약 |
|------|------|
| ✅ 효율 유입 | 검색(organic), 직접방문(direct)은 ROI가 높음 → **유지·강화 대상** |
| ❌ 비효율 유입 | 외부 referral 중 일부는 ROI가 매우 낮음 → **개선 or 예산 축소 대상** |
| 💡 시사점 | 광고 예산은 단순 유입수가 아닌 **성과 기준**으로 재조정해야 함 |



📌 [주제1] 예산 대비 총성과 대비 분석
예산 대비 얻은 전체 pageviews, hits, visits의 총합을 비교해, 효율적인 투자 채널을 객관적으로 구분할 수 있습니다.

In [9]:
# 성과 총합과 평균 효율 비교
top_avg = top10['pageviews_per_1000won'].mean()
bottom_avg = bottom10['pageviews_per_1000won'].mean()
ratio = round(top_avg / bottom_avg, 2) if bottom_avg > 0 else "Infinity"

top_sum = top10['pageviews'].sum()
bottom_sum = bottom10['pageviews'].sum()

print(f"✅ Top10 평균 효율성: {top_avg:.3f} / Bottom10 평균: {bottom_avg:.6f} → {ratio}배 차이")
print(f"📊 Top10 총 pageviews: {top_sum}, Bottom10: {bottom_sum}")


✅ Top10 평균 효율성: 1.449 / Bottom10 평균: 0.000808 → 1794.05배 차이
📊 Top10 총 pageviews: 318744, Bottom10: 240


📌 [주제2] 예산은 높은데 성과는 낮은 ‘비효율 투자 후보군’ 탐색
예산은 상위지만, 효율은 하위인 비정상 트래픽을 찾아내서 개선/중단 판단 근거 제공

In [10]:
# 예산 상위 20% & 효율 하위 20% 교집합 추출
budget_threshold = agg_df_sm['budget'].quantile(0.8)
eff_threshold = agg_df_sm['pageviews_per_1000won'].quantile(0.2)

inefficient_candidates = agg_df_sm[
    (agg_df_sm['budget'] >= budget_threshold) &
    (agg_df_sm['pageviews_per_1000won'] <= eff_threshold)
][['source', 'medium', 'budget', 'pageviews_per_1000won']].sort_values(by='budget', ascending=False)

inefficient_candidates


,source,medium,budget,pageviews_per_1000won
11,duckduckgo.com,referral,46088382,0.000694
33,pinterest.com,referral,45089344,0.000732
43,trainup.withgoogle.com,referral,43209280,0.001342
35,quora.com,referral,38413597,0.000469
